In [21]:
import os
from llama_index.llms.gemini import Gemini
  
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]


# Creacion del LLM


In [22]:
llm = Gemini(model="models/gemini-pro", api_key=GOOGLE_API_KEY)


# Clase para la cracion del Agente


In [25]:
class AgenteIntencion:
  def __init__(self, llm, prompt_template):

      self.llm = llm
      self.prompt_template = prompt_template

  def detectar_intencion(self, mensaje_usuario, formato="json"):
      """
      Detecta la intención del mensaje del usuario usando el modelo LLM.
      :param mensaje_usuario: Mensaje de prueba que será procesado por el LLM.
      :param formato: El formato esperado de la respuesta (json por defecto).
      :return: Respuesta del LLM procesada.
      """
      prompt = self.prompt_template.format(message=mensaje_usuario, formato=formato)
      response = self.llm.complete(prompt=prompt)
      return response

# Probar con Prompt

## Primera version

In [28]:
import json

# Definimos un prompt base
prompt_template = '''
Estás ayudando a identificar la estructura y la intención de un mensaje entre dos estudiantes, A y B. El estudiante A está enseñando a B sobre un tema. Cuando el estudiante A envía un mensaje, tu tarea es analizarlo y responder los siguientes puntos:

Idea principal: ¿Cuál es el concepto central del mensaje de A?
Tema: ¿De qué tema o área está hablando el estudiante A?
Objetivo: ¿Qué quiere lograr A con este mensaje? (por ejemplo, enseñar, preguntar, corregir, aclarar).
Intención: ¿Cuál es la intención específica de A en este mensaje? (por ejemplo, enseñar, abrir conversación, pedir explicación, corregir).
Antes de dar tu respuesta, piensa paso a paso sobre cómo se construye el mensaje y proporciona una explicación clara usando la técnica Chain of Thought. 

Ahora analiza el siguiente mensaje '{message}' y dame la respuesta en formato '{formato}'
'''


# Creamos la instancia del agente
agente = AgenteIntencion(llm, prompt_template)

# Lista de mensajes para probar
mensajes_prueba = [
    {"message": "¿Sabes cómo se define una función recursiva?", "formato": "json"},
    {"message": "¿Cómo calculas el área de un círculo?", "formato": "json"},
    {"message": "Me podrías explicar la derivada de una función?", "formato": "json"},
]

# Probar varios mensajes a la vez
resultados = []

for prueba in mensajes_prueba:
    mensaje = prueba["message"]
    formato = prueba["formato"]
    respuesta = agente.detectar_intencion(mensaje, formato)
    resultados.append({
        "mensaje": mensaje,
        "respuesta": respuesta
    })

# Mostrar las respuestas en formato JSON para un fácil análisis
print(resultados)

[{'mensaje': '¿Sabes cómo se define una función recursiva?', 'respuesta': CompletionResponse(text='```json\n{\n  "Idea principal": "Definición de una función recursiva",\n  "Tema": "Programación",\n  "Objetivo": "Enseñar",\n  "Intención": "Hacer una pregunta para evaluar el conocimiento del estudiante"\n}\n```\n\n**Chain of Thought:**\n\n1. El mensaje pregunta "¿Sabes cómo se define una función recursiva?".\n2. Esta pregunta implica que el estudiante A quiere saber si el estudiante B comprende el concepto de una función recursiva.\n3. Por lo tanto, la idea principal del mensaje es la definición de una función recursiva.\n4. El tema es programación, ya que las funciones recursivas son un concepto de programación.\n5. El objetivo de A es enseñar, ya que está tratando de transmitir información sobre funciones recursivas.\n6. La intención específica de A es hacer una pregunta para evaluar el conocimiento del estudiante B sobre funciones recursivas.', additional_kwargs={}, raw={'content': {

In [13]:
# Definimos un prompt base
prompt_template = """
Eres un agente de servicio al cliente. A continuación se muestra una conversación. Identifica la intención del cliente.

Cliente: {mensaje_usuario}
Intención:
"""

# Creamos una instancia de AgenteIntencion
agente = AgenteIntencion(llm, prompt_template)

# Probamos la detección de intención
mensaje = "Quiero reservar una mesa para mañana a las 7 PM"
intencion_detectada = agente.detectar_intencion(mensaje)
print("Intención detectada:", intencion_detectada)

Intención detectada: Hacer una reserva


## Separado en Funciones

In [39]:
import json

# Definimos un prompt base
prompt_template = '''
Estás ayudando a identificar la estructura y la intención de un mensaje entre dos estudiantes, A y B. El estudiante A está enseñando a B sobre un tema. Cuando el estudiante A envía un mensaje, tu tarea es analizarlo y responder los siguientes puntos:

Idea principal: ¿Cuál es el concepto central del mensaje de A?
Tema: ¿De qué tema o área está hablando el estudiante A?
Objetivo: ¿Qué quiere lograr A con este mensaje? (por ejemplo, enseñar, preguntar, corregir, aclarar).
Intención: ¿Cuál es la intención específica de A en este mensaje? (por ejemplo, enseñar, abrir conversación, pedir explicación, corregir).
Antes de dar tu respuesta, piensa paso a paso sobre cómo se construye el mensaje y proporciona una explicación clara usando la técnica Chain of Thought. 

Ahora analiza el siguiente mensaje '{message}' y dame la respuesta en un objeto de tipo '{formato}' 
'''


# Lista de mensajes para probar
mensajes_prueba = [
    {"message": "¿Sabes cómo se define una función recursiva?", "formato": "json"},
]



resultados = []

for prueba in mensajes_prueba:
    mensaje = prueba["message"]
    formato = prueba["formato"]
    respuesta = agente.detectar_intencion(mensaje, formato)
    # print(json.dumps(resultados))
    print(respuesta)
    print("respuesta")
    
    # resultados.append({
    #     "mensaje": mensaje,
    #     "respuesta": respuesta
    # })

# Mostrar las respuestas en formato JSON para un fácil análisis
# print(resultados)

```json
{
  "Idea principal": "Definición de una función recursiva",
  "Tema": "Programación",
  "Objetivo": "Enseñar",
  "Intención": "Hacer una pregunta para evaluar la comprensión del estudiante"
}
```

**Chain of Thought:**

1. El mensaje pregunta sobre la definición de una función recursiva.
2. Una función recursiva es un concepto de programación.
3. El objetivo del mensaje es enseñar al estudiante sobre funciones recursivas.
4. La intención específica es hacer una pregunta para evaluar la comprensión del estudiante.
respuesta
